In [2]:
import numpy as np
import pandas as pd
from pandas import Series ,DataFrame

file_path="azureml://subscriptions/4f93438e-847c-469d-94ae-ef5ee130ccab/resourcegroups/project_team14/workspaces/mlnotebook/datastores/testml/paths/Final_Data/part-00000-tid-1107757838746000038-dd6987f9-5526-4378-b9fd-633c8f0e2565-1388-1-c000.csv"

In [3]:
df=pd.read_csv(file_path,low_memory=False) 

In [4]:
df["Reported Date"] = pd.to_datetime(df["Reported Date"])

In [7]:
df2 = df.copy()

In [6]:
garr = df2.groupby("Commodity").agg(avg_arrival=('Arrivals (Tonnes)', 'mean')).round(2)

tarr = garr['avg_arrival'].sum()

avgarr = garr.assign(percentage=lambda x: (x['avg_arrival'] / tarr) * 100)

ranks = avgarr.sort_values(by='percentage', ascending=False)

print(ranks)

                     avg_arrival  percentage
Commodity                                   
GladiolusCutFlower        695.04    6.146037
Jaffri                    681.43    6.025688
RoseLocal                 660.00    5.836189
Lilly                     550.00    4.863491
Marget                    542.52    4.797347
...                          ...         ...
Maragensu                   1.00    0.008843
RatTailRadishMogari         1.00    0.008843
Lukad                       0.63    0.005571
Gramflour                   0.43    0.003802
Jamamkhan                   0.24    0.002122

[297 rows x 2 columns]


In [8]:
newcols = ranks.head(25)
newcols

,avg_arrival,percentage
Commodity,,
GladiolusCutFlower,695.04,6.146037
Jaffri,681.43,6.025688
RoseLocal,660.00,5.836189
Lilly,550.00,4.863491
Marget,542.52,4.797347
TenderCoconut,342.72,3.030574
RoseLoose,269.17,2.380192
Hen,248.86,2.200597
Bamboo,237.50,2.100144


In [9]:
cols =['GladiolusCutFlower', 'Jaffri', 'RoseLocal', 'Lilly', 'Marget',
       'TenderCoconut', 'RoseLoose', 'Hen', 'Bamboo', 'TubeRoseSingle',
       'ChrysanthemumLoose', 'BroomstickFlowerBroom', 'Carnation',
       'PaddyDhanBasmati', 'TubeFlower', 'Egg', 'Cock', 'CottonSeed',
       'SkinAndHide', 'Coconut', 'Wood', 'Cotton', 'DailaChandni',
       'MyrobolanHarad', 'TubeRoseLoose']
top = newcols.index.tolist()

df2 = df2[df2['Commodity'].isin(top)].copy()

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error

Shape of Sampled DataFrame: (120655, 11)


In [20]:
from sklearn.preprocessing import LabelEncoder

columns_to_encode = ['Commodity', 'State Name', 'District Name', 'Market Name', 'Variety', 'Group']

label_encoder = LabelEncoder()

for column in columns_to_encode:
    df2[column] = label_encoder.fit_transform(df2[column])

print(df2)


          Commodity  State Name  District Name  Market Name  Variety  Group  \
1415951          20          11             91          359      120      5   
1415952          20          11             91          359      120      5   
1415953          20          12            143          174      141      5   
1415954          20          12            143          174      141      5   
1415955          20          12             67          267      141      5   
...             ...         ...            ...          ...      ...    ...   
49957190         11          19             87          416       18      6   
49957191         11          20             95          372       17      6   
49957192         11          13            203         1082       17      6   
49957193         11          13            203         1082       17      6   
49957194         11          13            203         1082       17      6   

          Arrivals (Tonnes)  Min Price (Rs/Quintal)

InvalidParameterError: The 'loss' parameter of GradientBoostingRegressor must be a str among {'quantile', 'absolute_error', 'squared_error', 'huber'}. Got 'ls' instead.

In [29]:
df2.head()

,Commodity,State Name,District Name,Market Name,Variety,Group,Arrivals (Tonnes),Min Price (Rs/Quintal),Max Price (Rs/Quintal),Modal Price (Rs/Quintal),Reported Date
1415951,20,11,91,359,120,5,34.0,600.0,650.0,650.0,2003-04-04
1415952,20,11,91,359,120,5,40.0,650.0,700.0,700.0,2003-04-02
1415953,20,12,143,174,141,5,400.0,2000.0,2500.0,2300.0,2003-04-04
1415954,20,12,143,174,141,5,200.0,2000.0,2300.0,2100.0,2003-04-01
1415955,20,12,67,267,141,5,500.0,2600.0,2600.0,2600.0,2003-04-03


In [31]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error

target_variable = 'Modal Price (Rs/Quintal)'
df2 = df2.drop(columns=['Reported Date'])

X = df2.drop(columns=[target_variable])
y = df2[target_variable]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

gb_regressor = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, loss='squared_error')  # You can adjust these hyperparameters as needed

gb_regressor.fit(X_train, y_train)

gb_predictions = gb_regressor.predict(X_test)

mae_gb = mean_absolute_error(y_test, gb_predictions)
print(f'Mean Absolute Error (Gradient Boosting): {mae_gb}')

param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7]
}

grid_search = GridSearchCV(GradientBoostingRegressor(), param_grid, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_

print("Best Hyperparameters:", best_params)

best_model = grid_search.best_estimator_
best_predictions = best_model.predict(X_test)

mae_best_model = mean_absolute_error(y_test, best_predictions)
print(f'Mean Absolute Error (Best Model): {mae_best_model}')


Mean Absolute Error (Gradient Boosting): 160.8588003354565


In [ ]:
import os
import pickle
from azureml.core import Workspace

#Azure Machine Learning workspace
ws = Workspace.from_config()

file_path1 = 'models/rf_state.pkl'
#file_path1 = 'azureml://subscriptions/4f93438e-847c-469d-94ae-ef5ee130ccab/resourcegroups/project_team14/workspaces/mlnotebook/datastores/testml/paths/models/rf_state.pkl'

with open(file_path1, 'wb') as f:
    pickle.dump(gb_regressor, f)
default_datastore = ws.get_default_datastore()
default_datastore.upload_files(files=['models/gb_regressor.pkl'], target_path='models', overwrite=True)

default_datastore.register_model(model_name='gb_regressor', model_path='models/gb_regressor.pkl')

In [ ]:
 
import pickle 
  
# Saving trained model as a pickle string. 
saved_model = pickle.dumps(gb_regressor)